# Árvore Patrícia

**Apresentação para o seminário da disciplina Projeto de Algoritmos II**

## Integrantes

- Eduardo Gil Serrão Cardoso
- Gabriela Souza Maximino
- Igor Matheus Souza Moreira
- Marco Aurélio Lima do Nascimento Júnior

## Agenda

- [Introdução](#Introdução)
    - [Pesquisa Digital](#Pesquisa-Digital)
    - [O que é PATRICIA?](#O-que-é-a-PATRICIA?)
- [A árvore PATRICIA](#A-árvore-PATRICIA)
    - [Visão Geral](#Visão-Geral)
    - [Disclaimer](#Disclaimer)
    - [Definições e Propriedades](#Definições-e-Propriedades)
        - [Classe Node](#Classe-Node)
        - [Classe Arvore](#Classe-Arvore)
    - [Na prática](#Na-prática)
        - [Exemplo de uso](#Exemplo-de-uso)
        - [Teste de confiabilidade](#Teste-de-confiabilidade)
- [Complexidade das operações](#Complexidade-das-operações)
- [Comparação com outras estruturas de dados](#Comparação-com-outras-estruturas-de-dados)
- [Aplicações](#Aplicações)
- [Conclusões]()
- [Referências]()

## Introdução

- Tem-se uma maior capacidade de armazenamento em dispositivos de memória (primária e secundária);
- Arquivos de grande conteúdo (que estão na RAM) puderam ser manipulados;


- **Problema:** como recuperar uma informação em meio a tantas dentro desse arquivo?
- **Solução:** métodos de pesquisa: sequencial, binária… e **pesquisa digital!**

[Voltar ao topo](#Árvore-Patrícia)

### Pesquisa Digital

- Arquivos são divididos em registros. Cada registro possui uma chave;
- O **objetivo das pesquisas** em geral é encontrar, baseado em uma chave de busca, chaves iguais no arquivo;
- A busca digital consiste em representar essas chaves como dígitos, podendo ser divisíveis;
- A estrutura básica para uma pesquisa digital é a árvore digital;
- As árvores digitais também são conhecidas como Trie (re**TRIE**val);
- A **árvore PATRICIA** é uma árvore digital!

[Voltar ao topo](#Árvore-Patrícia)

### O que é a PATRICIA?

- Acrônimo para ***P*** *ractical* ***A*** *lgorithm* ***t*** *o* ***R*** *etrieve* ***I*** *nformation* ***C*** *oded* ***i*** *n* ***A*** *lphanumeric* (Algoritmo Prático para Obter Informação Codificada em Alfanumérico);
- Criada por Donald R. Morrison em 1968 (referência abaixo);
- Implementação eficiente da árvore digital binária.

Morrison, D. R. (1968). PATRICIA—practical algorithm to retrieve information coded in alphanumeric. Journal of the ACM (JACM), 15(4), 514-534.

[Voltar ao topo](#Árvore-Patrícia)

## A árvore PATRICIA

### Visão Geral

Usaremos as sete palavras **casco**, **cascalho**, **cascata**, **dama**, **domando**, **dominar** e **domínio** para representar as árvores.

- **Árvore Trie:** cada nível representa um caractere
    - **Problema:** gera muitos nós.
    
![](imagens/exemplo_de_trie.png)
    
- **Árvore Radix:** árvore de prefixos.
    - **Melhoria:** diminui o número de nós criados.
    
![](imagens/exemplo_de_radix.png)
    
- **Árvore Patrícia:** uma árvore Trie compacta... em algumas definições.

![](imagens/exemplo_de_patricia.png)

[Voltar ao topo](#Árvore-Patrícia)

### *Disclaimer*

Algumas definições diferentes foram encontradas:
- PATRICIA enquanto árvore Trie compacta;
- PATRICIA enquanto variação da árvore Radix 2 (Radix binária).

Optamos por nos ater à definição de árvore PATRICIA como variação de árvore Trie.


[Voltar ao topo](#Árvore-Patrícia)

### Definições e Propriedades

- Estritamente binária;
- Filhos unários de nós são compactados com seus respectivos pais;
- Cada nó guarda a posição da chave a partir da qual as sub-árvores começam a se diferenciar, possibilitando as comparações;
- As chaves são guardadas apenas nos nós-folha.


[Voltar ao topo](#Árvore-Patrícia)

#### Classe `Node`

In [1]:
class Node:

    def __init__(self, pos, val, prefixo):

        """
        Unidade basica
        :param int pos:
        :param str val:
        :param str prefixo:
        """

        self.filhos = [None, None]

        self.prefixo = prefixo
        
        self.pos = pos
        
        self.val = ord(val)

    def get(self, palavra):

        if palavra[:self.pos] != self.prefixo or len(palavra) <= self.pos: return 0

        valor = ord(palavra[self.pos])

        if valor <= self.val:

            return self.filhos[0]

        return self.filhos[1]


    def derivados(self):

        derivados = []

        for i in self.filhos:

            if type(i) == str: derivados.append(i[:-1])

            else: derivados += i.derivados()

        return derivados


    def __repr__(self):

        return "[{}|{}]".format(self.pos, chr(self.val))

[Voltar ao topo](#Árvore-Patrícia)

#### Classe `Arvore`

In [2]:
class Arvore:

    def __init__(self, raiz):

        """
        Inicia a árvore
        :param str raiz:
        """

        self.raiz = raiz + "$"


    def get_intersect(self, p1, p2):

        """
        Buca o ponto onde duas strings de diferenciam
        :param str p1:
        :param str p2:
        :return int:
        """
        for i in range(len(p1)):

            if len(p2) > i:

                if p1[i] != p2[i]: return i


    def insere(self, palavra):

        """
        Insere uma string na árvore
        :param str palavra:
        :return None:
        """

        # print("Inserindo %s" % palavra)

        if self.check(palavra):

            print("[Erro] Palavra já inserida")
            
            return

        palavra += '$'

        pai = None
        
        node = self.raiz
        
        i = 0

        while 1:

            # Caso nó folha

            if type(node) == str:

                pos = self.get_intersect(palavra, node)
                
                # Caso nó folha é raiz também

                if pai is None:

                    l = sorted([(ord(palavra[pos]), palavra), (ord(node[pos]),node)])
                    
                    self.raiz = Node(pos, chr(l[0][0]), l[0][1][:pos])
                    
                    self.raiz.filhos = [k[1] for k in l]

                    return


                # Caso padrão

                else:

                    lado = pai.filhos.index(node)

                    l = sorted([(ord(palavra[pos]), palavra), (ord(node[pos]),node)])

                    pai.filhos[lado] = Node(pos, chr(l[0][0]), l[0][1][:pos])
                    
                    pai.filhos[lado].filhos = [k[1] for k in l]

                    return



            else:

                prox = node.get(palavra)

                # Caso prefixo não combina

                if not prox:

                    pos = self.get_intersect(palavra, node.prefixo)

                    l = sorted([(ord(palavra[pos]), palavra), (ord(node.prefixo[pos]),node)])

                    # Caso raiz

                    if pai is None:

                        self.raiz = Node(pos, chr(l[0][0]), palavra[:pos])

                        self.raiz.filhos = [k[1] for k in l]

                        return

                    else:

                        lado = pai.filhos.index(node)
                        
                        pai.filhos[lado] = Node(pos, chr(l[0][0]), palavra[:pos])
                        
                        pai.filhos[lado].filhos = [k[1] for k in l]

                        return

                # Base de iteração

                else:

                    pai = node
                    
                    node = prox


    def check(self, palavra):

        palavra += '$'

        node = self.raiz

        while 1:

            if type(node) == str:

                return node == palavra

            next = node.get(palavra)
            
            if next:

                node = next

            else:

                return False


    def remove(self, palavra):

        if not self.check(palavra):

            print("[Erro] Palavra não encontrada")
            return

        palavra += '$'

        avo = None
        pai = None
        node = self.raiz

        while 1:

            if type(node) == str:

                if pai is None:

                    print("[Erro] Não é possível remover único item da arvore")
                    
                    return

                else:

                    pai.filhos.remove(node)
                    
                    novo_sucessor = pai.filhos[0]

                    if avo is None:

                        self.raiz = novo_sucessor
                        
                        del pai

                        return

                    else:

                        index = avo.filhos.index(pai)
                        
                        avo.filhos[index] = novo_sucessor
                        
                        del pai

                        return

            else:

                prox = node.get(palavra)

                if pai is None:

                    pai = node
                    
                    node = prox

                else:

                    avo = pai
                    
                    pai = node
                    
                    node = prox

    def derivados(self, prefixo):

        """
        Exibe folhas derivadas de determinado prefixo
        :param str prefixo:
        :return str[]:
        """

        tam = len(prefixo)
        
        node = self.raiz

        while 1:

            if type(node) == str:

                if node[:tam] == prefixo:

                    return [node[:-1]]

                else:

                    return []

            else:

                if node.prefixo == prefixo or node.prefixo[:tam] == prefixo:

                    return node.derivados()

                next = node.get(prefixo)

                if next:
                    
                    node = next
                    
                else:
                    
                    return []

[Voltar ao topo](#Árvore-Patrícia)

### Na prática

#### Exemplo

Realizaremos o mesmo exemplo exposto anteriormente, só que na prática.

In [43]:
tree = Arvore("casco")
tree.raiz

'casco$'

Ao visualizar a raíz, notamos a presença de uma string. Isso quer dizer que estamos em um nó folha, fato este representado pelo $\$$ que finaliza a palavra.

In [46]:
type(tree.raiz)

str

Naturalmente, pelo fato desta árvore conter apenas uma palavra, não há uso de objetos `Node`, tampouco há a presença de filhos:

In [48]:
try:
    tree.raiz.filhos
except AttributeError:
    print("Não há filhos!")

Não há filhos!


In [ ]:
# tree.raiz.filhos

A inserção de mais uma palavra fará com que o objeto `str` seja substituído por um `Node`, e que a profundidade da árvore aumente.

In [49]:
tree.insere("cascalho")
tree.raiz

[4|a]

In [50]:
tree.raiz.filhos

['cascalho$', 'casco$']

Sigamos adiante com a adição das demais palavras:

In [51]:
tree.insere("cascata")
tree.insere("dama")
tree.insere("domando")
tree.insere("dominar")
tree.insere("domínio")

Vejamos como a árvore está em relação à imagem vista anteriormente, replicada abaixo por conveniência:

![](imagens/exemplo_de_patricia.png)

**Profundidade 0:**

In [52]:
tree.raiz

[0|c]

**Profundidade 1:**

In [53]:
tree.raiz.filhos

[[4|a], [1|a]]

**Profundidade 2:**

In [54]:
(tree.raiz.filhos[0].filhos + 
 tree.raiz.filhos[1].filhos)

[[5|l], 'casco$', 'dama$', [3|a]]

Para efeito ilustrativo, adicionaremos a partir daqui espaços vazios quando houver folhas (objetos `Node` sem filho)

**Profundidade 3:**

In [55]:
(tree.raiz.filhos[0].filhos[0].filhos + 
 [""] +
 [""] +
 [""] +
 [""] +
 tree.raiz.filhos[1].filhos[1].filhos)

['cascalho$', 'cascata$', '', '', '', '', 'domando$', [3|i]]

**Profundidade 4:**

In [56]:
([""] +
 [""] +
 [""] +
 [""] +
 [""] +
 [""] +
 [""] +
 [""] +
 [""] +
 [""] +
 [""] +
 [""] +
 [""] +
 [""] +
 tree.raiz.filhos[1].filhos[1].filhos[1].filhos)

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'dominar$',
 'domínio$']

[Voltar ao topo](#Árvore-Patrícia)

#### Teste de confiabilidade

A fim de garantir a confiabilidade da estrutura de dados, fizemos um teste intensivo de inserção de palavras, valendo-nos do dicionário em português do Brasil fornecido pelo LibreOffice.

In [76]:
from random import shuffle

In [77]:
def load_dict(path):
    with open(path, "r", encoding="utf-8") as file:
        return [line.strip("\n") for line in file.readlines()]

Carreguemos o dicionário:

In [78]:
pt_br = load_dict("./dictionaries/palavras.txt")

In [96]:
print(f"O dicionário possui {len(pt_br)} palavras.")

O dicionário possui 320139 palavras.


O teste consiste na inserção, verificação e remoção de palavras de forma aleatória. Dessa forma, precisamos de três cópias do dicionário:

In [79]:
load   = pt_br.copy()
check  = pt_br.copy()
remove = pt_br.copy()

In [80]:
print(f"load[:5]:   {load[:5]}")
print(f"check[:5]:  {check[:5]}")
print(f"remove[:5]: {remove[:5]}")

load[:5]:   ['a', 'ª', 'à', 'á', 'ã']
check[:5]:  ['a', 'ª', 'à', 'á', 'ã']
remove[:5]: ['a', 'ª', 'à', 'á', 'ã']


Vamos utilizar a função `shuffle`, do módulo `random`, para obter versões aleatorizadas do dicionário:

In [90]:
shuffle(load)
shuffle(check)
shuffle(remove)

In [91]:
print(f"load[:5]:   {load[:5]}")
print(f"check[:5]:  {check[:5]}")
print(f"remove[:5]: {remove[:5]}")

load[:5]:   ['diftógrafo', 'cientifização', 'manolita', 'xantematina', 'ácoros-bastardos']
check[:5]:  ['pau-morcego', 'executado', 'catartínico', 'coptotermes', 'alveoliforme']
remove[:5]: ['circungirar', 'colossigmoidostômico', 'francoáceo', 'ofiuroideo', 'asquistítico']


Por fim, seguiremos adiante com o teste propriamente dito. Basicamente, a expectativa é que a primeira palavra inserida seja a primeira da lista `load` e termine com a primeira palavra de `remove`.

In [92]:
tree = Arvore(load[0])
tree.raiz

'diftógrafo$'

In [93]:
print("Inserindo...")

for word in load[1:]:
    # print(f"Inserting {word}...")
    tree.insere(word)
    if not tree.check(word):
        raise ValueError(f"Não pude encontrar a palavra {word}!")

print("Verificando...")

for word in check:
    # print(f"Verifying {word}...")
    if not tree.check(word):
        raise ValueError(f"Não pude verificar que a palavra {word} está mesmo na árvore!")

print("Removendo...")

for word in remove[1:]:
    # print(f"Removing {word}...")
    tree.remove(word)
    if tree.check(word):
        raise ValueError(f"Não consegui remover a palavra {word}!")

print("Sucesso.")

Inserindo...
Verificando...
Removendo...
Sucesso.


In [95]:
tree.raiz

'circungirar$'

[Voltar ao topo](#Árvore-Patrícia)

## Complexidade das operações

- **Busca:** $O(|chave|)$;
- **Inserção:** $O(|chave| + |X|)$;
- **Remoção:** $O(|chave| + |X|)$;
- **Listagem de todos os registros que contêm determinado prefixo:** $O(|s| + k)$.

[Voltar ao topo](#Árvore-Patrícia)

## Comparação com outras estruturas de dados

- Compacta em relação a Trie

[Voltar ao topo](#Árvore-Patrícia)

## Aplicações

- Armazenamento de dados no Ethereum;
- Bloqueio de IPs pelo Sqreen;
- Indexação de arquivos XML.

[Voltar ao topo](#Árvore-Patrícia)

## Fim.

**Perguntas?**